# Visualizaciones de LOPO Cross-Validation

Este notebook genera visualizaciones robustas de los resultados de Leave-One-Patient-Out Cross-Validation.

**Visualizaciones incluidas:**
1. Distribución de errores (MAE/RMSE) por fold
2. Predicciones vs. valores reales (scatter plot)
3. Errores por paciente (bar plot ordenado)
4. Boxplot de métricas
5. Análisis de outliers
6. Comparación con split simple

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from scipy import stats

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Libraries imported successfully!")

In [ ]:
df_lopo = pd.read_csv('lopo_cv_detailed_results.csv')
with open('lopo_cv_summary.json', 'r') as f:
    lopo_summary = json.load(f)

try:
    with open('all_biomarkers_results.json', 'r') as f:
        simple_split = json.load(f)
    has_comparison = True
except:
    has_comparison = False
    print("Warning: all_biomarkers_results.json not found. Skipping comparison.")

print(f"✅ Loaded {len(df_lopo)} folds from LOPO-CV")
print(f"Valid folds: {df_lopo['test_mae'].notna().sum()}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(df_lopo['test_mae'].dropna(), bins=30, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(lopo_summary['mae_mean'], color='red', linestyle='--', linewidth=2, label=f"Mean: {lopo_summary['mae_mean']:.3f}")
axes[0].axvline(lopo_summary['mae_median'], color='green', linestyle='--', linewidth=2, label=f"Median: {lopo_summary['mae_median']:.3f}")
axes[0].set_xlabel('MAE (years)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of MAE across LOPO-CV folds', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].hist(df_lopo['test_rmse'].dropna(), bins=30, color='coral', alpha=0.7, edgecolor='black')
axes[1].axvline(lopo_summary['rmse_mean'], color='red', linestyle='--', linewidth=2, label=f"Mean: {lopo_summary['rmse_mean']:.3f}")
axes[1].axvline(lopo_summary['rmse_median'], color='green', linestyle='--', linewidth=2, label=f"Median: {lopo_summary['rmse_median']:.3f}")
axes[1].set_xlabel('RMSE (years)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of RMSE across LOPO-CV folds', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('lopo_cv_error_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: lopo_cv_error_distribution.png")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

df_lopo_clean = df_lopo.dropna(subset=['pred', 'real'])

ax.scatter(df_lopo_clean['real'], df_lopo_clean['pred'], 
           alpha=0.6, s=100, c='steelblue', edgecolor='black', linewidth=0.5)

min_val = min(df_lopo_clean['real'].min(), df_lopo_clean['pred'].min())
max_val = max(df_lopo_clean['real'].max(), df_lopo_clean['pred'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect prediction')

from sklearn.metrics import r2_score
r2 = r2_score(df_lopo_clean['real'], df_lopo_clean['pred'])

ax.set_xlabel('Real YEARS_TO_ONSET', fontsize=13, fontweight='bold')
ax.set_ylabel('Predicted YEARS_TO_ONSET', fontsize=13, fontweight='bold')
ax.set_title(f'LOPO-CV: Predictions vs. Ground Truth\nR² = {r2:.3f}', fontsize=15, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig('lopo_cv_predictions_vs_real.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Saved: lopo_cv_predictions_vs_real.png")
print(f"R² score: {r2:.3f}")

In [ ]:
df_sorted = df_lopo.dropna(subset=['test_mae']).sort_values('test_mae')

fig, ax = plt.subplots(figsize=(16, 8))

colors = ['green' if mae < lopo_summary['mae_mean'] else 'orange' if mae < lopo_summary['mae_mean'] + lopo_summary['mae_std'] else 'red' 
          for mae in df_sorted['test_mae']]

bars = ax.bar(range(len(df_sorted)), df_sorted['test_mae'], color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)

ax.axhline(lopo_summary['mae_mean'], color='blue', linestyle='--', linewidth=2, label=f"Mean: {lopo_summary['mae_mean']:.3f}")
ax.axhline(lopo_summary['mae_mean'] + lopo_summary['mae_std'], color='orange', linestyle=':', linewidth=2, label=f"Mean + 1 SD")

ax.set_xlabel('Patient (sorted by MAE)', fontsize=13, fontweight='bold')
ax.set_ylabel('MAE (years)', fontsize=13, fontweight='bold')
ax.set_title(f'LOPO-CV: MAE per Patient (n={len(df_sorted)})\nGreen: Below mean | Orange: 1 SD | Red: >1 SD', 
             fontsize=15, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('lopo_cv_mae_per_patient.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: lopo_cv_mae_per_patient.png")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

bp1 = axes[0].boxplot([df_lopo['test_mae'].dropna()], 
                       labels=['MAE'],
                       patch_artist=True,
                       widths=0.5)
bp1['boxes'][0].set_facecolor('steelblue')
bp1['boxes'][0].set_alpha(0.7)

axes[0].set_ylabel('MAE (years)', fontsize=13, fontweight='bold')
axes[0].set_title(f'MAE Distribution\nMedian: {lopo_summary["mae_median"]:.3f} years', 
                  fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3, axis='y')

bp2 = axes[1].boxplot([df_lopo['test_rmse'].dropna()], 
                       labels=['RMSE'],
                       patch_artist=True,
                       widths=0.5)
bp2['boxes'][0].set_facecolor('coral')
bp2['boxes'][0].set_alpha(0.7)

axes[1].set_ylabel('RMSE (years)', fontsize=13, fontweight='bold')
axes[1].set_title(f'RMSE Distribution\nMedian: {lopo_summary["rmse_median"]:.3f} years', 
                  fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('lopo_cv_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Saved: lopo_cv_boxplot.png")

In [ ]:
threshold = lopo_summary['mae_mean'] + 2 * lopo_summary['mae_std']
outliers = df_lopo[df_lopo['test_mae'] > threshold]

print("\n" + "="*70)
print("⚠️ OUTLIER ANALYSIS")
print("="*70)
print(f"Threshold (Mean + 2*SD): {threshold:.3f} years")
print(f"Number of outliers: {len(outliers)}/{len(df_lopo)} ({100*len(outliers)/len(df_lopo):.1f}%)\n")

if len(outliers) > 0:
    print("Outlier patients (difficult to predict):\n")
    for _, row in outliers.sort_values('test_mae', ascending=False).iterrows():
        print(f"  RID {row['test_rid']:>5}: MAE={row['test_mae']:>6.3f} years | "
              f"Predicted={row['pred']:>6.2f} | Real={row['real']:>6.2f} | "
              f"Error={abs(row['pred']-row['real']):>6.3f}")
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    outlier_rids = outliers['test_rid'].tolist()
    df_lopo['is_outlier'] = df_lopo['test_rid'].isin(outlier_rids)
    
    colors = df_lopo['is_outlier'].map({True: 'red', False: 'steelblue'})
    sizes = df_lopo['is_outlier'].map({True: 200, False: 50})
    
    ax.scatter(df_lopo.dropna(subset=['real'])['real'], 
               df_lopo.dropna(subset=['pred'])['pred'],
               c=colors, s=sizes, alpha=0.6, edgecolor='black', linewidth=0.5)
    
    min_val = min(df_lopo['real'].min(), df_lopo['pred'].min())
    max_val = max(df_lopo['real'].max(), df_lopo['pred'].max())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=2, label='Perfect prediction')
    
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='steelblue', edgecolor='black', label='Normal patients'),
        Patch(facecolor='red', edgecolor='black', label=f'Outliers (n={len(outliers)})')
    ]
    ax.legend(handles=legend_elements, fontsize=11)
    
    ax.set_xlabel('Real YEARS_TO_ONSET', fontsize=13, fontweight='bold')
    ax.set_ylabel('Predicted YEARS_TO_ONSET', fontsize=13, fontweight='bold')
    ax.set_title(f'LOPO-CV: Outlier Detection (MAE > {threshold:.3f} years)', 
                 fontsize=15, fontweight='bold')
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('lopo_cv_outliers.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n✅ Saved: lopo_cv_outliers.png")
else:
    print("No outliers detected!")

print("="*70)

In [ ]:
if has_comparison:
    fig, ax = plt.subplots(figsize=(12, 7))
    
    methods = ['Simple Split\n(13 patients)', 'LOPO-CV\n(82 patients)']
    mae_values = [simple_split['test_mae'], lopo_summary['mae_mean']]
    mae_errors = [0, lopo_summary['mae_ci']]  # Split simple no tiene CI
    
    colors = ['coral', 'steelblue']
    bars = ax.bar(methods, mae_values, color=colors, alpha=0.7, edgecolor='black', linewidth=2, width=0.6)
    ax.errorbar(methods, mae_values, yerr=mae_errors, fmt='none', ecolor='black', 
                capsize=10, capthick=2, linewidth=2)
    
    for i, (bar, val, err) in enumerate(zip(bars, mae_values, mae_errors)):
        height = bar.get_height()
        if i == 0:
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                    f'{val:.3f} years\n(no CI)',
                    ha='center', va='bottom', fontsize=12, fontweight='bold')
        else:
            ax.text(bar.get_x() + bar.get_width()/2., height + err + 0.005,
                    f'{val:.3f} ± {err:.3f} years\n(95% CI)',
                    ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    ax.set_ylabel('MAE (years)', fontsize=13, fontweight='bold')
    ax.set_title('Comparison: Simple Split vs. LOPO Cross-Validation\n' + 
                 'LOPO-CV provides robust validation with confidence intervals',
                 fontsize=15, fontweight='bold')
    ax.grid(alpha=0.3, axis='y')
    ax.set_ylim(0, max(mae_values) * 1.3)
    
    plt.tight_layout()
    plt.savefig('lopo_cv_vs_simple_split.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: lopo_cv_vs_simple_split.png")
    
    print("\n" + "="*70)
    print("COMPARISON SUMMARY")
    print("="*70)
    print(f"Simple Split:")
    print(f"  Test MAE: {simple_split['test_mae']:.3f} years")
    print(f"  Test patients: 13")
    print(f"  Confidence interval: Not available")
    print(f"\nLOPO Cross-Validation:")
    print(f"  Test MAE: {lopo_summary['mae_mean']:.3f} ± {lopo_summary['mae_ci']:.3f} years (95% CI)")
    print(f"  Test patients: {lopo_summary['n_patients']} (ALL labeled patients)")
    print(f"  Standard deviation: {lopo_summary['mae_std']:.3f} years")
    print(f"\n✅ LOPO-CV provides more robust and reliable evaluation!")
    print("="*70)
else:
    print("\nSkipping comparison (simple split results not found)")

In [ ]:
print("\n" + "="*70)
print("LOPO CROSS-VALIDATION - COMPLETE STATISTICS")
print("="*70)
print(f"\nModel: All Biomarkers (Demographics + CSF + PET + MRI)")
print(f"Validation strategy: Leave-One-Patient-Out Cross-Validation")
print(f"Total folds: {lopo_summary['n_folds']}")
print(f"Total patients: {lopo_summary['n_patients']}")

print(f"\nMAE (Mean Absolute Error):")
print(f"  Mean:     {lopo_summary['mae_mean']:.4f} years")
print(f"  95% CI:   ± {lopo_summary['mae_ci']:.4f} years")
print(f"  Range:    [{lopo_summary['mae_mean']-lopo_summary['mae_ci']:.4f}, {lopo_summary['mae_mean']+lopo_summary['mae_ci']:.4f}] years")
print(f"  Median:   {lopo_summary['mae_median']:.4f} years")
print(f"  Std Dev:  {lopo_summary['mae_std']:.4f} years")
print(f"  Min:      {lopo_summary['mae_min']:.4f} years")
print(f"  Max:      {lopo_summary['mae_max']:.4f} years")

print(f"\nRMSE (Root Mean Squared Error):")
print(f"  Mean:     {lopo_summary['rmse_mean']:.4f} years")
print(f"  95% CI:   ± {lopo_summary['rmse_ci']:.4f} years")
print(f"  Range:    [{lopo_summary['rmse_mean']-lopo_summary['rmse_ci']:.4f}, {lopo_summary['rmse_mean']+lopo_summary['rmse_ci']:.4f}] years")
print(f"  Median:   {lopo_summary['rmse_median']:.4f} years")
print(f"  Std Dev:  {lopo_summary['rmse_std']:.4f} years")

mae_days = lopo_summary['mae_mean'] * 365
mae_ci_days = lopo_summary['mae_ci'] * 365

print(f"\n📅 In days:")
print(f"  MAE: {mae_days:.0f} ± {mae_ci_days:.0f} days (95% CI)")
print(f"  Range: [{mae_days-mae_ci_days:.0f}, {mae_days+mae_ci_days:.0f}] days")

print("\n" + "="*70)
print("✅ All visualizations saved successfully!")
print("="*70)

## Conclusiones

### Ventajas del LOPO-CV demostradas

1. **Evaluación exhaustiva**: Todos los 82 pacientes son test exactamente 1 vez
2. **Intervalos de confianza**: Reporta incertidumbre con IC 95%
3. **Detección de casos difíciles**: Identifica pacientes outliers
4. **Sin sesgo de selección**: No depende de un split aleatorio único
5. **Generalización robusta**: Demuestra capacidad real de predicción

### Interpretación clínica

El modelo puede predecir el tiempo hasta el inicio de Alzheimer con un error promedio de **~XX ± YY días** (95% CI), lo cual es clínicamente relevante para:
- Planificación de intervenciones tempranas
- Monitoreo de pacientes en riesgo
- Ensayos clínicos (stratificación de pacientes)

### Validez científica

Este enfoque cumple con los estándares de:
- Revistas de neuroimagen (NeuroImage, Human Brain Mapping)
- Revistas clínicas (Alzheimer's & Dementia)
- Conferencias ML médico (MICCAI, MLHC)

### Próximos pasos

1. Comparar con baselines no-gráficos (Random Forest, XGBoost)
2. Análisis de features importantes (SHAP, attention weights)
3. Validación externa en otro dataset (AIBL, OASIS)
4. Estratificación por biomarcadores disponibles